# EROSIÓN EN NUBES DE PUNTOS

Este es un corto tutorial para explicar paso a paso la erosión morfologica aplicada a nubes de puntos acorde el trabajo:

- Balado, J., Van Oosterom, P., Díaz-Vilariño, L., & Meijers, M. (2020). Mathematical morphology directly applied to point cloud data. ISPRS Journal of Photogrammetry and Remote Sensing, 168, 208-220.

# Importación de librerías

Las librerías empleadas en la morfología matemática son *numpy* y *o3d*.

- https://numpy.org/
- http://www.open3d.org/

In [ ]:
import numpy as np
import open3d as o3d

# Lectura de datos

Aunque existen numerosos formatos de nubes de puntos, en este tutorial vamos a partir de nubes en formato *txt*, que pueden ser leídas mediante *numpy*, puesto que no tienen ningún tipo de compresión. La nube en *txt* se estructura en 1 punto por fila y un atributo por columna, además se especifica ' ' como delimitador entre columnas. En este caso, la morfología matemática solo trabaja sobre datos XYZ. 

Como ejemplo de trabajo se proporciona una nube en formato cubo abierto.


In [ ]:
# Lectura de datos
input_data = np.loadtxt("Nubes/cubo3d.txt", delimiter=' ')

# Visualización de nubes de puntos de entrada

Para visualizar los datos de entrada en 3D, recurrimos a la librería *o3d*, para lo cual es necesario transformar previamente nuestra nube de puntos a un objeto nube de puntos de la librería:

* Nota: los ejes predefinidos de la visualización no se corresponden con los ejes reales de la nube


* Nota: Para los no familiarizados con el acceso a los datos de nubes de puntos, tanto en *numpy* como en otras librerias, el acceso se realiza mediante [n filas, n columnas], de tal forma que con ":" se indica que se seleccionan todas las filas o columnas, y numéricamente mediante "n:m" se indica que se accede desde la fila/columna "n+1" a la m.

In [ ]:
# Crear objeto nube de puntos
pcd_in = o3d.geometry.PointCloud()

# Cargar puntos en el objeto, delimitado a XYZ en caso de que la nube tenga más atributos
pcd_in.points = o3d.utility.Vector3dVector(input_data[:,0:3])

# Visualización
o3d.visualization.draw_geometries([pcd_in])

# Definir Elemento Estructurante (SE)

El elemento estructurante (SE) es una nube de puntos que queremos combinar con la nube de entrada para modificarla, bien mediante operaciones de dilatación o de erosión. Por lo tanto, el SE solo contiene coordenadas XYZ. El primer punto del SE define el centro de translación del SE a lo largo de la nube de puntos de entrada. El resto de los puntos indican la dirección de dilatación o erosión. Definir el SE correctamente es la mayor complicación de la morfología matemática e implica un proceso de prueba y error.

<center> <img src="Figures/F01.jpg"></center>
<center> Figura 1. Ejemplos de SE </center>

En este ejemplo definimos un SE de tres puntos separados 0.1 metros en la dirección Z y centrado en [0 0 0], Figure 1.b.

In [ ]:
# Definir SE
SE =  np.array([[0, 0, 0],
      [0, 0, -0.1],
      [0, 0, 0.1]])

# Definir distancias de búsqueda

A diferencia de una imagen, cuya distribución de pixeles es regular, en las nubes de puntos la distancia entre puntos varía, por lo que es necesario indicar un radio de búsqueda de puntos para establecer proximidades. 

En el trabajo original, la distancia de búsqueda era calculada automáticamente en el proceso como la media de la distancia entre puntos vecinos. No obstante, la definición de la distancia manualmente da más libertad para operaciones de detección de objetos, segmentación y rellenado de huecos. 

Como distancia a indicar se propone la distancia entre puntos del SE, o la distancia entre puntos del objeto a dilatar/erosionar.

In [ ]:
# Definir radio de búsqueda
d = 0.1

# Erosionar

El proceso de erosión es, a grandes rasgos, un proceso de eliminación de puntos de la nube que no coincidan con el SE. Se basa en los siguientes pasos: 

1. Trasladar el SE a cada punto de la nube. La traslación del SE se hace tomando como referencia el primer punto del SE.
2. Comprobar si para cada punto del SE existe un punto cercano de la nube de entrada. El cálculo de proximidad se hace en librería *o3d* para calcular distancias entre nubes y se emplea *d* para identificar aquellos puntos lejanos.
3. Para conservar un punto de la nube de entrada, todos los puntos del SE trasladado deben estar próximos a otros puntos. El resultado de la comprobación de proximidad se guarda en una lista de índices y se actualiza para cada punto de la nube/SE.

<center> <img src="Figures/F03.jpg"></center>
<center> Figura 2. Proceso de erosión </center>

Se realizaron dos modificaciones respecto al trabajo original para optimizar computacionalmente el algoritmo. 

- El recorrido de los puntos se realiza sobre el SE y no sobre la nube de entrada, por lo que se realizan menos bucles. En este caso, en vez de trasladar el SE a cada punto de la nube, se traslada un punto del SE a todos los puntos de la nube. 
- En el algoritmo original se elimina el punto que está siendo evaluado en cada iteración si cumple los criterios de erosión. En esta versión, dado que todos los puntos de la nube de entrada se evalúan simultáneamente, se genera una lista de índices que se actualiza con información de proximidad de cada punto del SE en cada iteración.


In [ ]:
# Convertir la nube de entrada en objeto-nube (si no se hizo para visualización)
pcd_in = o3d.geometry.PointCloud()
pcd_in.points = o3d.utility.Vector3dVector(input_data[:,0:3])

# Generar índices de puntos a conservar
idx_remain = np.ones(input_data.shape[0], dtype=bool)

for i in range(1,SE.shape[0]):
    
    # Trasladar el punto i del SE a toda la nube
    translated_SE = input_data[:,0:3] + SE[i,:]
    
    # Convertir a SE trasladado a objeto-nube
    pcd_tSE = o3d.geometry.PointCloud()
    pcd_tSE.points = o3d.utility.Vector3dVector(translated_SE)
    
    # Calcular distancias entre nubes
    dist_pcd_tSE_2_pcd_in = pcd_tSE.compute_point_cloud_distance(pcd_in)
    dist_pcd_tSE_2_pcd_in = np.asarray(dist_pcd_tSE_2_pcd_in)    
    
    # Filtrado por distancias de puntos de la nube de entrada que coinciden con puntos del SE
    idx_aux = dist_pcd_tSE_2_pcd_in < d
    
    # Combinación con la lista de puntos anterior
    idx_remain = idx_remain * idx_aux

# Selección de los puntos de salida acorde a los indices de los puntos a conservar
output_data = input_data[idx_remain,0:3]

# Visualización de la nube de puntos erosionada

Para visualizar la nube erosionada en 3D, recurrimos a la librería *o3d*, para lo cual es necesario transformar nuevamente nuestros puntos de salida a un objeto nube de puntos de la librería *o3d*:

* Nota: los ejes predefinidos de la visualización no se corresponden con los ejes reales de la nube


In [ ]:
# Crear objeto nube de puntos
pcd_out = o3d.geometry.PointCloud()

# Cargar puntos en el objeto
pcd_out.points = o3d.utility.Vector3dVector(output_data)

# Visualización
o3d.visualization.draw_geometries([pcd_out])

# Exportar nube de puntos erosionada

Para guardar la nube en disco, se recurre a la librería *numpy* y a un formato de la nube en *txt*. Para el guardado se indica una dirección y nombre del archivo a generar.

In [ ]:
# Definicion de la ruta y nombre del archivo
ruta = "Nubes/erosionada.txt"

# Guardado
np.savetxt(ruta,output_data,delimiter=' ') 

Con esto concluye el tutorial de erosión morfológica. La aplicación de la erosión morfológica combinada con la dilatación morfológica en operaciones de apertura morfológica (para segmentación de objetos) y cierre morfológico (para el rellenado de huecos) se presentará en el siguiente tutorial.